#### This section relies on the CellOracle package for scanning scATAC-seq peak calls (TSS annotated) for TF binding site motifs. For further information, please refer to the CellOracle documentation available at https://morris-lab.github.io/CellOracle.documentation/index.html.

### Import library

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm
import celloracle as co
from celloracle import motif_analysis as ma
from celloracle.utility import save_as_pickled_object
co.__version__

'0.14.0'

In [3]:
# config InlineBackend.figure_format = 'retina'
# matplotlib inline

plt.rcParams['figure.figsize'] = (15,7)
plt.rcParams["savefig.dpi"] = 600

### Load reference genome and data

In [4]:
# PLEASE make sure reference genome is correct.
ref_genome = "hg19"

genome_installation = ma.is_genome_installed(ref_genome=ref_genome)
print(ref_genome, "installation: ", genome_installation)
print(ma.SUPPORTED_REF_GENOME)

if not genome_installation:
    import genomepy
    genomepy.install_genome(name=ref_genome, provider="UCSC")
else:
    print(ref_genome, "is installed.")


# Load annotated peak data.
peaks = pd.read_csv("~/Desktop/Buenrostro/res_Buenrostro2018_processed_peak_file.csv", index_col=0)
print(peaks.head())


genome hg19 is not installed in this environment.
Please install genome using genomepy.
e.g.
    >>> import genomepy
    >>> genomepy.install_genome(name="hg19", provider="UCSC")
hg19 installation:  False
               species            ref_genome           provider
0                Human                  hg38               UCSC
1                Human                  hg19               UCSC
2                Mouse                  mm39               UCSC
3                Mouse                  mm10               UCSC
4                Mouse                   mm9               UCSC
5         S.cerevisiae               sacCer2               UCSC
6         S.cerevisiae               sacCer3               UCSC
7            Zebrafish               danRer7               UCSC
8            Zebrafish              danRer10               UCSC
9            Zebrafish              danRer11               UCSC
10  Xenopus tropicalis               xenTro2               UCSC
11  Xenopus tropicalis     

15:13:43 | INFO | Downloading genome from UCSC. Target URL: https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/chromFa.tar.gz...


Download:   0%|          | 0.00/905M [00:00<?, ?B/s]

15:28:21 | INFO | Genome download successful, starting post processing...
15:29:49 | INFO | name: hg19
15:29:49 | INFO | local name: hg19
15:29:49 | INFO | fasta: /home/bio/.local/share/genomes/hg19/hg19.fa


Filtering Fasta: 0.00 lines [00:00, ? lines/s]

                     peak_id gene_short_name
0  chr10_100027770_100028555           LOXL4
1  chr10_100174589_100175172         PYROXD2
2  chr10_100175241_100175630         PYROXD2
3  chr10_100205646_100207085            HPS1
4  chr10_100205646_100207085    LOC101927278


In [5]:
def decompose_chrstr(peak_str):
    """
    Args:
        peak_str (str): peak_str. e.g. 'chr1_3094484_3095479'
        
    Returns:
        tuple: chromosome name, start position, end position
    """
    
    *chr_, start, end = peak_str.split("_")
    chr_ = "_".join(chr_)
    return chr_, start, end

from genomepy import Genome

def check_peak_format(peaks_df, ref_genome):
    """
    Check peak format. 
     (1) Check chromosome name. 
     (2) Check peak size (length) and remove sort DNA sequences (<5bp)
    
    """
    
    df = peaks_df.copy()
    
    n_peaks_before = df.shape[0]
    
    # Decompose peaks and make df
    decomposed = [decompose_chrstr(peak_str) for peak_str in df["peak_id"]]
    df_decomposed = pd.DataFrame(np.array(decomposed), index=peaks_df.index)
    df_decomposed.columns = ["chr", "start", "end"]
    df_decomposed["start"] = df_decomposed["start"].astype(int)
    df_decomposed["end"] = df_decomposed["end"].astype(int)
    
    # Load genome data
    genome_data = Genome(ref_genome)
    all_chr_list = list(genome_data.keys())
    
    
    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])
    
    
    # Filter peaks with invalid chromosome name
    n_threshold = 5
    df = df[(lengths >= n_threshold) & df_decomposed.chr.isin(all_chr_list)]
    
    # DNA length check
    lengths = np.abs(df_decomposed["end"] - df_decomposed["start"])
    
    # Data counting
    n_invalid_length = len(lengths[lengths < n_threshold])
    n_peaks_invalid_chr = n_peaks_before - df_decomposed.chr.isin(all_chr_list).sum()
    n_peaks_after = df.shape[0]
    
    
    #
    print("Peaks before filtering: ", n_peaks_before)
    print("Peaks with invalid chr_name: ", n_peaks_invalid_chr)
    print("Peaks with invalid length: ", n_invalid_length)
    print("Peaks after filtering: ", n_peaks_after)
    
    return df

In [6]:
peaks = check_peak_format(peaks, ref_genome)
print(peaks)

# Instantiate TFinfo object
tfi = ma.TFinfo(peak_data_frame=peaks, ref_genome=ref_genome) 
# print(tfi)

##time
# Scan motifs. !!CAUTION!! This step may take several hours if you have many peaks!
# import faulthandler
# faulthandler.enable()
tfi.scan(fpr=0.02, motifs=None, verbose=True)

Peaks before filtering:  21823
Peaks with invalid chr_name:  0
Peaks with invalid length:  0
Peaks after filtering:  21823
                         peak_id gene_short_name
0      chr10_100027770_100028555           LOXL4
1      chr10_100174589_100175172         PYROXD2
2      chr10_100175241_100175630         PYROXD2
3      chr10_100205646_100207085            HPS1
4      chr10_100205646_100207085    LOC101927278
...                          ...             ...
21818       chrY_2802833_2804297             ZFY
21819       chrY_2870572_2871816       LINC00278
21820       chrY_4868210_4868444         PCDH11Y
21821       chrY_6778489_6779751           TBL1Y
21822       chrY_7141392_7142656            PRKY

[21823 rows x 2 columns]
No motif data entered. Loading default motifs for your species ...
 Default motif for vertebrate: gimme.vertebrate.v5.0. 
 For more information, please see https://gimmemotifs.readthedocs.io/en/master/overview.html 

Initiating scanner... 



DEBUG:gimme.scanner:using background: genome hg19 with size 200


Calculating FPR-based threshold. This step may take substantial time when you load a new ref-genome. It will be done quicker on the second time. 



2023-11-07 15:35:22,777 - INFO - determining FPR-based threshold
INFO:gimme.scanner:determining FPR-based threshold


Motif scan started .. It may take long time.



scanning:   0%|          | 0/19766 [00:00<?, ? sequences/s]

DEBUG:gimme.scanner:Scanning


In [7]:
# Check motif scan results
print(tfi.scanned_df.head())

# Reset filtering
tfi.reset_filtering()

# Do filtering
tfi.filter_motifs_by_score(threshold=10)

# Format post-filtering results.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)

# Get final base GRN
df = tfi.to_dataframe()
print(df.head())

# Save result as a dataframe
df = tfi.to_dataframe()
df.to_parquet("~/Desktop/Buenrostro/res_Buenrostro2018_base_GRN_dataframe.parquet")

                     seqname           motif_id factors_direct  \
0  chr10_100027770_100028555  GM.5.0.Mixed.0001                  
1  chr10_100027770_100028555  GM.5.0.Mixed.0001                  
2  chr10_100027770_100028555  GM.5.0.Mixed.0001                  
3  chr10_100027770_100028555  GM.5.0.Mixed.0001                  
4  chr10_100027770_100028555  GM.5.0.Mixed.0001                  

  factors_indirect     score  pos  strand  
0        EGR1, SRF  8.468539  391       1  
1        EGR1, SRF  8.255221  392       1  
2        EGR1, SRF  8.009193  407       1  
3        EGR1, SRF  7.870592  373       1  
4        EGR1, SRF  7.684773  266      -1  
Filtering finished: 11398604 -> 2197747
1. Converting scanned results into one-hot encoded dataframe.


  0%|          | 0/19756 [00:00<?, ?it/s]

2. Converting results into dictionaries.


  0%|          | 0/17907 [00:00<?, ?it/s]

  0%|          | 0/1095 [00:00<?, ?it/s]

                     peak_id gene_short_name  9430076C15RIK  AC002126.6  \
0  chr10_100027770_100028555           LOXL4            0.0         0.0   
1  chr10_100174589_100175172         PYROXD2            0.0         0.0   
2  chr10_100175241_100175630         PYROXD2            0.0         0.0   
3  chr10_100205646_100207085            HPS1            0.0         0.0   
4  chr10_100205646_100207085    LOC101927278            0.0         0.0   

   AC012531.1  AC226150.2  AFP  AHR  AHRR  AIRE  ...  ZNF784  ZNF8  ZNF816  \
0         0.0         0.0  0.0  0.0   0.0   0.0  ...     0.0   0.0     0.0   
1         0.0         0.0  0.0  0.0   0.0   0.0  ...     0.0   0.0     0.0   
2         0.0         0.0  0.0  0.0   0.0   0.0  ...     0.0   0.0     0.0   
3         0.0         0.0  0.0  0.0   0.0   0.0  ...     0.0   0.0     0.0   
4         0.0         0.0  0.0  0.0   0.0   0.0  ...     0.0   0.0     0.0   

   ZNF85  ZSCAN10  ZSCAN16  ZSCAN22  ZSCAN26  ZSCAN31  ZSCAN4  
0    0.0      0.